# 李鲁鲁老师的Python课程 第三课 键盘钢琴

项目主页: https://github.com/LC1332/Learn-Python-with-GPT

- 建议老师尽量在这节课配置耳机，不然课堂可能会有一点点吵
- [ ] 尝试编写钢琴的程序
- [ ] 分析自动生成的代码， 尝试跑通代码
- [ ] py文件的运行
- [ ] 改为实时运行

这节课让我们来做个还挺有趣的应用

这个应用在之前的课程设计里面是没有的，我们在之前的课程设计里面，避免了向同学讲解完整的pygame游戏的写法

因为完全凭空手写一个pygame游戏，对于初学python的同学来说，是相对比较困难的。

但是在Copilot-Python课里面，你只需要向ChatGPT描述需求就可以进行编写。

## 让我们开始吧

我们试着在ChatGPT里面给出这样的prompt

```
我希望实现一个pygame程序，

当按下1、2、3、4、5的时候

会发出C、D、E、F、G的声音，持续0.5秒
```

In [1]:
import pygame
import time

# 初始化pygame
pygame.init()

# 创建窗口
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("Pygame Sound Example")

# 加载声音文件
sounds = {
    pygame.K_1: pygame.mixer.Sound("C.wav"),
    pygame.K_2: pygame.mixer.Sound("D.wav"),
    pygame.K_3: pygame.mixer.Sound("E.wav"),
    pygame.K_4: pygame.mixer.Sound("F.wav"),
    pygame.K_5: pygame.mixer.Sound("G.wav")
}

# 主循环
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key in sounds:
                sounds[event.key].play()
                time.sleep(0.5)  # 播放0.5秒
                sounds[event.key].stop()  # 停止播放

# 退出pygame
pygame.quit()


pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


FileNotFoundError: No such file or directory.

: 

这里我们发现了几个问题

- 在notebook环境下运行pygame是不合适的 很容易发生卡顿，
- 我们并没有CDE这些wav的文件，会报错。

显然第二个问题是更紧要的

不过计算机是可以模拟出单频率的声音的，

让我们来修改下刚才的prompt

```
我希望实现一个pygame程序，

当按下1、2、3、4、5的时候

会发出C、D、E、F、G的对应频率的声音，持续0.5秒

可以在初始化的时候先生成这些声音对应的wav文件
```

```python
import numpy as np
from scipy.io.wavfile import write
import pygame
import time

# 声音频率（Hz）
frequencies = {
    'C': 261.63,
    'D': 293.66,
    'E': 329.63,
    'F': 349.23,
    'G': 392.00
}

# 生成声音文件
def generate_sound(frequency, duration=0.5, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    wave = 0.5 * np.sin(2 * np.pi * frequency * t)
    return wave

# 保存wav文件
for note, freq in frequencies.items():
    sound_wave = generate_sound(freq)
    write(f"{note}.wav", 44100, sound_wave.astype(np.float32))

# 初始化pygame
pygame.init()

# 创建窗口
screen = pygame.display.set_mode((400, 300))
pygame.display.set_caption("Pygame Sound Example")

# 加载声音文件
sounds = {
    pygame.K_1: pygame.mixer.Sound("C.wav"),
    pygame.K_2: pygame.mixer.Sound("D.wav"),
    pygame.K_3: pygame.mixer.Sound("E.wav"),
    pygame.K_4: pygame.mixer.Sound("F.wav"),
    pygame.K_5: pygame.mixer.Sound("G.wav")
}

# 主循环
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key in sounds:
                sounds[event.key].play()
                time.sleep(0.5)  # 播放0.5秒
                sounds[event.key].stop()  # 停止播放

# 退出pygame
pygame.quit()
```

这个时候我们不适合在notebook里面直接运行这个程序了

李鲁鲁老师把这个程序保存在了pygame_codes/2_piano_v_0.py里面

这个时候我们可以打卡整个Learn-Python-with-GPT的目录

![Image](images/call_cmd.jpg)



然后在调出来的terminal里面 运行

```shell
python pygame_codes/2_piano_v_0.py
```

整个时候就会启动这个钢琴了

当然我们更建议在VSCode里面直接运行 这样调试起来也方便很多

如果cmd里面无法正常运行，就咨询老师 或者检查环境目录里面是不是正确指向了python的解释器

---

我们可以讨论一下，这个钢琴有什么可以改进的地方呢？

- 不能两个键同时按下
- 一个按键按下后必须等待0.5秒才能进行其他活动
- 音的持续长度不能控制
- 没有强弱
- 音色不好听

在这里很大程度上是因为我们的程序里面有一个time.sleep(0.5)

我们希望把这个特性去移除，这样我们前三个问题就可以被解决。

这个时候我们要修改我们程序的prompt


```
我希望实现一个pygame程序，

当按下1、2、3、4、5、6、7、8、9、0的时候

会发出C4、D4、E4、F4、G4、A4、B4、C5、D5、E5的对应频率的声音

可以在初始化的时候先在piano_sounds目录生成这些声音对应的wav文件

当按键按下时候，会播放对应的声音

如果按键抬起，则声音会停止。
```

我这里把生成的代码放在pygame_codes/2_piano_v_1.py里面了，你可以尝试运行这个程序来体验一下

整体来说这个程序分成了两个部分，一是生成声音，二是播放声音

生成声音的部分实际上和前面的程序时一样的，让我们来看播放的部分

```python
# 加载声音文件
sounds = {
    pygame.K_1: pygame.mixer.Sound("piano_sounds/C4.wav"),
    pygame.K_2: pygame.mixer.Sound("piano_sounds/D4.wav"),
    pygame.K_3: pygame.mixer.Sound("piano_sounds/E4.wav"),
    pygame.K_4: pygame.mixer.Sound("piano_sounds/F4.wav"),
    pygame.K_5: pygame.mixer.Sound("piano_sounds/G4.wav"),
    pygame.K_6: pygame.mixer.Sound("piano_sounds/A4.wav"),
    pygame.K_7: pygame.mixer.Sound("piano_sounds/B4.wav"),
    pygame.K_8: pygame.mixer.Sound("piano_sounds/C5.wav"),
    pygame.K_9: pygame.mixer.Sound("piano_sounds/D5.wav"),
    pygame.K_0: pygame.mixer.Sound("piano_sounds/E5.wav")
}

# 主循环
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key in sounds:
                sounds[event.key].play(-1)  # -1参数使声音循环播放
        elif event.type == pygame.KEYUP:
            if event.key in sounds:
                sounds[event.key].stop()  # 停止声音播放
```

这里涉及一个核心函数pygame.event.get() 这个会返回上一次调用之后 pygame获得到的键盘的各种事件

并且使用一个for循环进行了分析，并且能够用条件语句，判断每一个event对应了特定按钮的按下或者抬起

在传统的python教学下，我们需要花40分钟左右的时间，才能教会完整的这个程序

并且很大程度上，学生并没有能力去修改pygame的程序。但是在我们的Python-Copilot课程里面，却是可以考虑做这件事情的。

在这里，我们希望这节课做一个更开放的尝试

我们会给出几个大的修改方向，让同学自己尝试修改这个程序。

## 修改延时的设计

我们刚才看到一个按钮如果抬起，则这个声音会立刻停止

这使得有的音会持续很短的时间。

如果我希望一个按键按下时，能至少确保一个声音被演奏0.125秒，要怎么办呢？

## 更复杂的键盘布置

尝试增加更多的按键，包括半音，使得一些更复杂的乐曲能够被演奏

## 修改音色（自然）

查询不同乐器的音色（谐频分布），尝试让程序演奏接近某种乐器的电子音

# 修改音色（电子）

我们这里使用了纯频率音的方式，实际上在电子音发展的早期，有很多以前的音乐会使用三角波、方波等波形

尝试把wav替换为这些波形进行播放

## 方便和弦

电子琴能够方便地按出和弦伴奏

有乐理基础地同学，能否让程序实现简单地和弦

如果能在课堂上展示键盘弹唱 就更强了

## 可视化

为钢琴增加一定的可视化功能